In [ ]:
from photutils import DAOStarFinder, CircularAperture
from astropy.io import fits as fits
from astropy.stats import mad_std
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
frames_in_set = 10
object_sets = 12
star_sets = 6
dark_sets = 7

object_frames = object_sets * frames_in_set
star_frames = star_sets * frames_in_set
dark_frames = dark_sets * frames_in_set
flat_frames = 40

x = 1039
y = 1391

path_raw = 'C:/Users/Calum/Documents/My Documents/354_Eleonora_Flyby2/Actual_Data/Initial_Frames/'
path_clean = 'C:/Users/Calum/Documents/My Documents/354_Eleonora_Flyby2/Actual_Data/Cleaned_Frames/'
path_comb = 'C:/Users/Calum/Documents/My Documents/354_Eleonora_Flyby2/Actual_Data/Combined_Frames/'

In [ ]:
time = []

for j in range(object_sets):
    with fits.open(path_raw + 'object_' + str((j+1)*10-9) + '.fit') as hdul:
        hours = 3600 * int(hdul[0].header['DATE-OBS'][11:13])
        mins = 60 * int(hdul[0].header['DATE-OBS'][14:16])
        secs = int(hdul[0].header['DATE-OBS'][17:19])
        time_start = hours + mins + secs
        
    with fits.open(path_raw + 'object_' + str((j+1)*10-1) + '.fit') as hdul:
        hours = 3600 * int(hdul[0].header['DATE-OBS'][11:13])
        mins = 60 * int(hdul[0].header['DATE-OBS'][14:16])
        secs = int(hdul[0].header['DATE-OBS'][17:19])
        time_end = hours + mins + secs
        
    time.append((time_start + time_end) / 2)
    
time_begin = time[0]
for j in range(object_sets):
    time[j] -= time_begin

In [ ]:
ast_x = 386.61
ast_y = 589.04
coord_ast = np.sqrt(ast_x**2 + ast_y**2)
brightness_ast = []

for j in range(object_sets):
    with fits.open(path_comb + 'object' + '_' + str(j+1) + '.fit') as hdul:
        data = hdul[0].data
        
    FWHM = 10
    std = mad_std(data)
    daofind = DAOStarFinder(fwhm = FWHM, threshold = 22*std)
    sources = daofind(data)
    
    diff = 1000000
    if j == 6:
        ast_x = 345.91
        ast_y = 281.95
        coord_ast = coord_ast = np.sqrt(ast_x**2 + ast_y**2)
    for k in range(sources['flux'].size):
        coord = np.sqrt(sources['xcentroid'][k]**2 + sources['ycentroid'][k]**2)
        if abs(coord_ast - coord) < diff: 
            diff = abs(coord_ast - coord)
            index = k
            
    ast_x = sources['xcentroid'][index]
    ast_y = sources['ycentroid'][index]
    coord_ast = np.sqrt(ast_x**2 + ast_y**2)
    brightness_ast.append(sources['flux'][index])

In [ ]:
brightness_star = []

for j in range(star_sets):
    with fits.open(path_comb + 'star' + '_' + str(j+1) + '.fit') as hdul:
        data = hdul[0].data
        
    FWHM = 10
    std = mad_std(data)
    daofind = DAOStarFinder(fwhm = FWHM, threshold = 22*std)
    sources = daofind(data)
    max_flux = max(sources['flux'])
    
    diff = 1000000
    for k in range(sources['flux'].size):
        if abs(sources['flux'][k] - max_flux) < diff: 
            diff = abs(sources['flux'][k] - max_flux)
            index = k
            
    brightness_star.append(sources['flux'][index])

In [ ]:
comp_factor = []
for element in brightness_star:
    comp_factor.append(element/brightness_star[0])

brightness_corr = []
for j in range(star_sets):
    brightness_corr.append(brightness_ast[2*j]/comp_factor[j])
    brightness_corr.append(brightness_ast[2*j+1]/comp_factor[j])

In [ ]:
plt.subplot(211)
plt.plot(time, brightness_ast)
plt.title('Partial light curves of 354 Eleonora')

plt.subplot(212)
plt.plot(time, brightness_corr)